In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.a_weekly_demand_planning_dc_sku 
AS
WITH _demand AS (
    SELECT 
        calday, dc_id, product_id, assortment_type,
        COUNT(DISTINCT IF(mdq > 0, store_id, NULL)) cnt_store_active,
        COUNT(DISTINCT IF(block_reason IS NOT NULL, store_id, NULL)) cnt_store_block_kstk,
        concat_ws(',', collect_set(block_reason)) block_reason_kstk,
        COUNT(DISTINCT IF(is_problem_article = 'YES', store_id, NULL)) cnt_store_problem_article,
        COUNT(DISTINCT IF(bonus_buy_id IS NOT NULL, store_id, NULL)) cnt_store_block_promotion,
        SUM(IF(is_block = 1, 0, demand_store)) demand_store_with_block,
        SUM(demand_store) demand_store_without_block
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.a_weekly_demand_planning_store_sku
    GROUP BY 1,2,3,4
),
_dr AS (
    SELECT 
        calday + 1 calday, dc_id, product_id,
        SUM(dr_internal) / NULLIF(SUM(dr_count_internal), 0) AS dr_internal,
        --SAFE_DIVIDE(SUM(adj_dr_internal), SUM(dr_count_internal)) adj_dr_internal
        SUM(adj_dr_internal) / NULLIF(SUM(dr_count_internal), 0) AS adj_dr_internal

    FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.a_dc_store_nonfresh_daily
    GROUP BY 1,2,3
),
main AS (
    SELECT a.*, b.dr_internal, b.adj_dr_internal
    FROM _demand a
    LEFT JOIN _dr b
        USING(calday, dc_id, product_id)
)
SELECT *
FROM main;
""")